In [ ]:
from elasticsearch import Elasticsearch
import pandas as pd
import glob
import time
from elasticsearch import helpers
import numpy as np
import json
import os
from datetime import datetime

source_file = '../data/meetup_event_data_2017032712.txt`'

bulk_size = 1024
index_name = 'meetup-' + datetime.today().strftime('%y.%m.%d')
doc_type = 'meetup'
#op_type = 'create'


os.environ['TZ'] = 'US/Pacific'
print time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime())

es = Elasticsearch(['http://158.85.180.83:9200'], send_get_body_as='POST')

files = [source_file]

for file in files:
    print 'processing', file, '....'
    actions = []
    #df = pd.read_csv(f, sep=',', error_bad_lines=False)
    with open(file) as f:
            for line in f:
                #print line[:-3]
                action = json.loads(line[:-3])
                action['_index'] = index_name
                action['_type'] = doc_type
                action['@timestamp'] = datetime.now()
                actions.append(action)

    actions = np.array(actions)
    #print len(actions)
    #print actions[:3]
    for idx in range(len(actions)/bulk_size+1):
        idx_range = bulk_size*(idx+1) if idx<len(actions)/bulk_size else len(actions)
        print idx*bulk_size, idx_range
        helpers.bulk(es, actions[idx*bulk_size:idx_range])
    #helpers.bulk(es,actions)
    print time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime())